### Import Libraries
In this section, we import the necessary libraries for our data processing task.

In [2]:
from pyspark import SparkContext, SparkConf
import re
import math

### Create Cleaning Pipeline
This function initializes a data cleaning pipeline using Spark. It reads raw text data, groups lines into job postings, and applies text cleaning and field extraction. The pipeline processes the data in several steps:
- **Map to Job Postings**: Groups lines into individual job postings based on start and end markers.
- **Clean Text**: Converts text to lowercase, removes punctuation, and standardizes common terms.
- **Extract Fields**: Uses regular expressions to extract specific fields like job title, description, skills, and salary from each posting.
- **Remove Duplicates**: Ensures each job posting is unique by removing duplicates.
- **Calculate Statistics**: Computes statistics such as the total number of postings and those with missing values.

In [3]:
def create_cleaning_pipeline(sc, input_file):
    """
    Enhanced cleaning pipeline with better preprocessing
    """
    sc.setLogLevel("WARN")
    
    # Read the raw text file
    raw_data = sc.textFile(input_file, minPartitions=100)

    # Step 1: MAP - Group lines into job postings
    def map_to_job_postings(lines):
        current_posting = []
        for line in lines:
            if "<<<START_JOB_POSTING>>>" in line:
                if current_posting:
                    yield "\n".join(current_posting)
                current_posting = []
            elif "<<<END_JOB_POSTING>>>" in line:
                if current_posting:
                    yield "\n".join(current_posting)
                current_posting = []
            else:
                current_posting.append(line)
    
    job_postings = raw_data.mapPartitions(map_to_job_postings)
    
    # Step 2: MAP - Clean text and extract fields
    def clean_text(text):
        if not text:
            return ""
        text = text.lower()
        text = re.sub(r'[^\w\s]', ' ', text)
        replacements = {
            'yrs': 'years',
            'yr': 'year',
            'exp': 'experience',
            'min': 'minimum',
            'max': 'maximum'
        }
        for old, new in replacements.items():
            text = text.replace(old, new)
        return ' '.join(text.split())
    
    def extract_fields(posting):
        try:
            # Extract all required fields
            title_match = re.search(r'\n(.*?) at (.*?)\nPosted', posting)
            job_title = clean_text(title_match.group(1)) if title_match else None
            
            # Extract description (everything between ABOUT THE ROLE and REQUIREMENTS)
            desc_match = re.search(r'ABOUT THE ROLE\n-+\n(.*?)\n\nREQUIREMENTS', posting, re.DOTALL)
            job_description = clean_text(desc_match.group(1)) if desc_match else None
            
            # Extract skills
            skills_match = re.search(r'SKILLS NEEDED\n-+\n(.*?)\n\n', posting, re.DOTALL)
            job_skills = clean_text(skills_match.group(1)) if skills_match else None
            
            # Extract industry
            industry_match = re.search(r'"Industry":"([^"]+)"', posting)
            job_industry = clean_text(industry_match.group(1)) if industry_match else None
            
            # Extract location
            location_match = re.search(r'Location: (.*?)\n', posting)
            job_location = clean_text(location_match.group(1)) if location_match else None
            
            # Extract experience
            exp_match = re.search(r'Experience: (.*?)\n', posting)
            experience_level = clean_text(exp_match.group(1)) if exp_match else None
            
            # Extract salary range
            salary_match = re.search(r'Salary: (.*?)\n', posting)
            salary_str = salary_match.group(1) if salary_match else ""
            nums = re.findall(r'\d+', salary_str)
            min_salary = int(nums[0]) * 1000 if len(nums) >= 2 else None
            max_salary = int(nums[1]) * 1000 if len(nums) >= 2 else None
            
            # Create tuple with all fields
            result = (
                job_title,
                job_description,
                job_skills,
                job_industry,
                job_location,
                experience_level,
                min_salary,
                max_salary
            )
            
            # Check if any required fields are missing
            if any(x is None for x in result):
                return None
                
            return result
            
        except Exception as e:
            return None
    
    # Extract and clean fields
    cleaned_postings = job_postings.map(extract_fields).filter(lambda x: x is not None)
    
    # Remove duplicates based on all fields
    unique_postings = cleaned_postings.distinct()
    
    # Calculate statistics
    total_posts = unique_postings.count()
    posts_with_missing = cleaned_postings.filter(lambda x: None in x).count()
    
    # Calculate salary statistics
    salary_stats = unique_postings.map(
        lambda x: (x[6], x[7])  # min and max salary
    ).filter(
        lambda x: x[0] is not None and x[1] is not None
    ).mapValues(
        lambda x: (x, 1)
    ).reduceByKey(
        lambda x, y: (x[0] + y[0], x[1] + y[1])
    ).mapValues(
        lambda x: x[0] / x[1]  # average salary
    )
    
    print(f"\nProcessing Summary:")

    print_cleaning_statistics(raw_data, job_postings, cleaned_postings, unique_postings)
    
    return unique_postings.collect(), salary_stats.collect()


In [4]:
# Create a new Spark context
conf = SparkConf().setAppName("JobDataCleaning") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "2g") \
    .set("spark.default.parallelism", "100")
sc = SparkContext(conf=conf)

# Process data
input_file = "hdfs:///user/ubuntu/job_descriptions_unstructured.txt"
final_data, stats = create_cleaning_pipeline(sc, input_file)


In [1]:
def print_cleaning_statistics(raw_data, job_postings, cleaned_postings, unique_postings):

    print("\n=== Data Cleaning Statistics ===")
    print(f"Initial raw lines: {raw_data.count():,}")
    print(f"Initial job postings: {job_postings.count():,}")
    
    # Salary statistics
    salary_stats = unique_postings.filter(lambda x: x[6] is not None and x[7] is not None)
    salary_count = salary_stats.count()
    avg_min_salary = salary_stats.map(lambda x: x[6]).mean()
    avg_max_salary = salary_stats.map(lambda x: x[7]).mean()
    print(f"\nSalary Information:")
    print(f"Postings with salary information: {salary_count:,}")
    print(f"Average minimum salary: ${avg_min_salary:,.2f}")
    print(f"Average maximum salary: ${avg_max_salary:,.2f}")
    
    print("\n=== Processing Complete ===")

In [ ]:
# Define columns for the CSV header
columns = [
    "job_title",
    "job_description",
    "job_skills",
    "job_industry",
    "job_location",
    "experience_level",
    "min_salary",
    "max_salary"
]

# Save the cleaned data to CSV
def row_to_csv(row):
    # Convert each field to string and handle None values
    csv_fields = [str(field) if field is not None else '' for field in row]
    # Join fields with commas and properly escape any commas within fields
    return ','.join(f'"{field}"' for field in csv_fields)

# Create header
header = ','.join(columns)

### Write Data to CSV
This section handles the export of processed data to a CSV file. It writes the cleaned job data to a specified output path, ensuring that the data is saved in a structured format for further analysis or use.

In [ ]:
import os
import math
from datetime import datetime
    
def write_data_to_csv(sc, final_data, header, output_path, batch_size=5000):
    
    print(f"\nStarting CSV export at {datetime.now().strftime('%H:%M:%S')}")
    print(f"Total records to process: {len(final_data):,}")
    
    # Ensure output directory exists
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Remove existing file if it exists
    if os.path.exists(output_path):
        os.remove(output_path)
    
    # Write header directly to file
    with open(output_path, 'w') as f:
        f.write(header + '\n')
    
    # Process batches
    num_batches = math.ceil(len(final_data) / batch_size)
    print(f"Processing in {num_batches} batches of {batch_size} records each")
    
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = min((i + 1) * batch_size, len(final_data))
        batch_data = final_data[start_idx:end_idx]
        
        print(f"\nProcessing batch {i+1}/{num_batches} ({start_idx:,} to {end_idx:,} records)")
        
        # Convert batch to CSV format
        batch_rdd = sc.parallelize(batch_data).map(row_to_csv)
        
        # Save batch directly to main file
        batch_rows = batch_rdd.collect()
        with open(output_path, 'a') as f:
            f.write('\n'.join(batch_rows) + '\n')
        
        print(f"Batch {i+1} complete - {len(batch_rows):,} records written")
    
    print(f"\nCSV export completed at {datetime.now().strftime('%H:%M:%S')}")
    print(f"Data saved to: {output_path}")
    
    # Verify file
    with open(output_path, 'r') as f:
        total_lines = sum(1 for _ in f)
    print(f"Total lines in output file: {total_lines:,} (including header)")

def write_stats_to_csv(sc, stats, stats_header, output_path):
    
    print(f"\nProcessing salary statistics at {datetime.now().strftime('%H:%M:%S')}")
    
    # Ensure the output directory exists
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Remove existing file if it exists
    if os.path.exists(output_path):
        os.remove(output_path)
    
    # Format statistics with proper number formatting
    formatted_stats = [(company, f"{float(avg_salary):,.2f}") for company, avg_salary in stats]
    
    # Calculate some summary statistics
    salaries = [float(avg_salary) for _, avg_salary in formatted_stats]
    if salaries:
        avg_salary = sum(salaries) / len(salaries)
        min_salary = min(salaries)
        max_salary = max(salaries)
        
        print("\nSalary Statistics Summary:")
        print(f"Number of companies: {len(formatted_stats):,}")
        print(f"Average salary across all companies: ${avg_salary:,.2f}")
        print(f"Minimum average salary: ${min_salary:,.2f}")
        print(f"Maximum average salary: ${max_salary:,.2f}")
        
        # Show sample of data
        print("\nSample of company statistics (first 5):")
        for company, avg in formatted_stats[:5]:
            print(f"Company: {company}, Average Salary: ${float(avg):,.2f}")
    
    # Write to file
    with open(output_path, 'w') as f:
        f.write(stats_header + '\n')
        for company, avg in formatted_stats:
            f.write(f"{company},{avg}\n")
    
    print(f"\nStatistics saved to: {output_path}")
    print(f"Total companies in statistics file: {len(formatted_stats):,}")


In [ ]:
write_data_to_csv(sc, final_data, header, "cleaned_job_data_using_spark.csv")



Starting CSV export at 08:39:38
Total records to process: 1,610,355
Processing in 323 batches of 5000 records each

Processing batch 1/323 (0 to 5,000 records)


Batch 1 complete - 5,000 records written

Processing batch 2/323 (5,000 to 10,000 records)


Batch 2 complete - 5,000 records written

Processing batch 3/323 (10,000 to 15,000 records)


Batch 3 complete - 5,000 records written

Processing batch 4/323 (15,000 to 20,000 records)


Batch 4 complete - 5,000 records written

Processing batch 5/323 (20,000 to 25,000 records)


...

Batch 319 complete - 5,000 records written

Processing batch 320/323 (1,595,000 to 1,600,000 records)


Batch 320 complete - 5,000 records written

Processing batch 321/323 (1,600,000 to 1,605,000 records)


Batch 321 complete - 5,000 records written

Processing batch 322/323 (1,605,000 to 1,610,000 records)


Batch 322 complete - 5,000 records written

Processing batch 323/323 (1,610,000 to 1,610,355 records)


Batch 323 complete - 355 records written

CSV export completed at 08:52:57
Data saved to: cleaned_job_data_using_spark.csv
Total lines in output file: 1,610,356 (including header)


NameError: name 'stats_header' is not defined

In [ ]:
""" # Save statistics to a separate CSV
stats_header = "company,avg_salary"
stats_csv_rdd = sc.parallelize([stats_header]) \
    .union(sc.parallelize(stats).map(lambda x: f"{x[0]},{x[1]}"))
stats_csv_rdd.coalesce(1).saveAsTextFile("salary_stats.csv")

print(f"\nResults saved to:")
print(f"- Job data: {output_path}")
print(f"- Salary statistics: salary_stats.csv") """

: 

In [ ]:
+

SyntaxError: invalid syntax (15193383.py, line 1)